<a href="https://colab.research.google.com/github/nerfstudio-project/nerfstudio/blob/tancik%2Fpolycam/colab/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
    <picture>
    <source media="(prefers-color-scheme: light)" srcset="https://autonomousvision.github.io/sdfstudio/resources/sdf_studio_4.svg">
    <img alt="sdfstudio" src="https://autonomousvision.github.io/sdfstudio/resources/sdf_studio_4.svg" width="400">
    </picture>
</p>


# A Unified Framework for Surface Reconstruction

This colab shows how to train and view SDFStudio both on pre-made datasets or from your own videos/images/polycam scan.

\\

Credit to [NeX](https://nex-mpi.github.io/) for Google Colab format.

In [ ]:
#@markdown <h1>Install Conda (requires runtime restart)</h1>

!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
#@markdown <h1>Install SDFStudio and Dependencies (~15 min)</h1>

%cd /content/
!pip install --upgrade pip
!pip install timm pytorch_lightning torch==1.12.1+cu113 torchvision==0.13.1+cu113 -f https://download.pytorch.org/whl/torch_stable.html

# Installing TinyCuda
%cd /content/
!gdown "https://drive.google.com/u/1/uc?id=1q8fuc-Mqiev5GTBTRA5UPgCaQDzuqKqj" 
!pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch

# Installing COLMAP
%cd /content/
!conda install -c conda-forge colmap

# Install SDFStudio
%cd /content/
!git clone https://github.com/autonomousvision/sdfstudio.git
%cd sdfstudio
!pip install --upgrade pip setuptools
!pip install -e .
# install tab completion
!ns-install-cli

# Install omnidata and pretrained models
%cd /content/
!git clone https://github.com/EPFL-VILAB/omnidata.git
%cd /content/omnidata/omnidata_tools/torch
!mkdir -p pretrained_models 
!gdown '1Jrh-bRnJEjyMCS7f-WsaFlccfPjJPPHI&confirm=t' -O ./pretrained_models/ # omnidata depth (v2)
!gdown '1wNxVO4vVbDEMEpnAi_jwQObf2MFodcBR&confirm=t' -O ./pretrained_models/ # omnidata normals (v2)

In [ ]:
#@markdown <h1> Downloading and Processing Data</h1>
#@markdown <h3>Pick the demo data or upload your own images/video and process</h3>
import os
import glob
from google.colab import files
from IPython.core.display import display, HTML

scene = '\uD83D\uDCE4 upload your images' #@param ['💀 dtu-scan65', '🏠 replica-room0', '📤 upload your images' , '🎥 upload your own video', '🔺 upload Polycam data']
#@markdown <h3> Select scene type if you upload your own data</h3>
scene_type = "object" #@param ["indoor", "object", "unbound"] {allow-input: true}

scene = ' '.join(scene.split(' ')[1:])
sdf_data_dir = None

if scene in ['dtu-scan65', 'replica-room0']:
    %cd /content/sdfstudio
    !python scripts/downloads/download_data.py sdfstudio
    sdf_data_dir = f"/content/sdfstudio/data/sdfstudio-demo-data/{scene}"
else:
  data_type = None
  pre_data_dir = None
  if scene == "upload Polycam data":
      data_type = "polycam"
      %cd /content/
      !mkdir -p /content/data/nerfstudio/custom_data
      %cd /content/data/nerfstudio/custom_data/
      uploaded = files.upload()
      dir = os.getcwd()
      if len(uploaded.keys()) > 1:
          print("ERROR, upload a single .zip file when processing Polycam data")
      pre_data_dir = [os.path.join(dir, f) for f in uploaded.keys()][0]
  elif scene in ['upload your images', 'upload your own video']:
      data_type = "colmap"
      display(HTML('<h3>Select your custom data</h3>'))
      display(HTML('<p/>You can select multiple images by pressing ctrl, cmd or shift and click.<p>'))
      display(HTML('<p/>Note: This may take time, especially on hires inputs, so we recommend to download dataset after creation.<p>'))
      !mkdir -p /content/data/nerfstudio/custom_data
      if scene == 'upload your images':
          !mkdir -p /content/data/nerfstudio/custom_data/raw_images
          %cd /content/data/nerfstudio/custom_data/raw_images
          uploaded = files.upload()
          dir = os.getcwd()
      else:
          %cd /content/data/nerfstudio/custom_data/
          uploaded = files.upload()
          dir = os.getcwd()
      preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
      del uploaded
      %cd /content/

      pre_data_dir = "/content/data/nerfstudio/custom_data/"
      if scene == 'upload your images':
          !ns-process-data images --data /content/data/nerfstudio/custom_data/raw_images --output-dir $pre_data_dir
      else:
          video_path = preupload_datasets[0]
          !ns-process-data video --data $video_path --output-dir $pre_data_dir

  scene = "custom_data"
  sdf_data_dir = "/content/sdfstudio/data/custom_data"
  %cd /content/sdfstudio/
  !python scripts/datasets/process_nerfstudio_to_sdfstudio.py \
  --data-type $data_type --scene-type $scene_type \
  --data $pre_data_dir \
  --output-dir $sdf_data_dir

print("Data processing succeeded!")

In [ ]:
#@markdown <h1>Set up and Start Viewer</h1>

%cd /content

# Install localtunnel
# We are using localtunnel https://github.com/localtunnel/localtunnel but ngrok could also be used
!npm install -g localtunnel

# Tunnel port 7007, the default for
!rm url.txt 2> /dev/null
get_ipython().system_raw('lt --port 7007 >> url.txt 2>&1 &')

import time
time.sleep(3) # the previous command needs time to write to url.txt


with open('url.txt') as f:
  lines = f.readlines()
websocket_url = lines[0].split(": ")[1].strip().replace("https", "wss")
# from nerfstudio.utils.io import load_from_json
# from pathlib import Path
# json_filename = "nerfstudio/nerfstudio/viewer/app/package.json"
# version = load_from_json(Path(json_filename))["version"]
url = f"https://viewer.nerf.studio/?websocket_url={websocket_url}"
print(url)
print("You may need to click Refresh Page after you start training!")
from IPython import display
display.IFrame(src=url, height=800, width="100%")

In [ ]:
!conda remove --force qt-main

In [ ]:
#@markdown <h1>Start Training</h1>
%cd /content/sdfstudio/
!ns-train neus-facto --pipeline.model.sdf-field.inside-outside False \
--vis viewer --experiment-name neus-facto-$scene sdfstudio-data \
--data $sdf_data_dir --auto-orient True

In [ ]:
#@markdown <h1>Extract Mesh</h1>
config_file_path = "/content/sdfstudio/outputs/neus-facto-dtu-scan65/neus-facto/2023-01-17_200838/config.yml" #@param {type:"string"}
output_path = "./meshes/neus-facto-dtu65.ply" #@param {type:"string"}

%cd /content/sdfstudio
!ns-extract-mesh --load-config $config_file_path --output-path $output_path

In [ ]:
#@markdown <h1>Texture Mesh with Nerf</h1>
target_num_faces = 10000 #@param {type:"integer"}
config_file_path = "/content/sdfstudio/outputs/neus-facto-dtu-scan65/neus-facto/2023-01-17_200838/config.yml" #@param {type:"string"}
mesh_file_path = "meshes/neus-facto-dtu65.ply" #@param {type:"string"}
output_dir = "./textures" #@param {type:"string"}

%cd /content/sdfstudio/
!python scripts/texture.py --load-config $config_file_path --input-mesh-filename $mesh_file_path --output-dir $output_dir --target_num_faces $target_num_faces